## Task 2. Import and balance data
Import London addresses. Test the result.

In [2]:
!mongoimport  --host localhost:60000 --db taxiDb --collection londonPostcodes --type csv --file data/London_postcodes.csv  --headerline --ignoreBlanks -v

2022-01-04T16:30:56.555+0200	using write concern: &{majority false 0}
2022-01-04T16:30:56.559+0200	using 8 decoding workers
2022-01-04T16:30:56.559+0200	using 1 insert workers
2022-01-04T16:30:56.563+0200	will listen for SIGTERM, SIGINT, and SIGKILL
2022-01-04T16:30:56.601+0200	filesize: 174255090 bytes
2022-01-04T16:30:56.603+0200	using fields: Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,District Code,Ward Code,Country,County Code,Constituency,Introduced,Terminated,Parish,National Park,Population,Households,Built up area,Built up sub-division,Lower layer super output area,Rural/urban,Region,Altitude,London zone,LSOA Code,Local authority,MSOA Code,Middle layer super output area,Parish Code,Census output area,Constituency Code,Index of Multiple Deprivation,Quality,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income,Sewage Company,Travel To Work Area,ITL level

`mongos> db.stats()`<br>
{
	"raw" : {
		"shard_repl_set2/shard2_node1:27017,shard2_node2:27017,shard2_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 0,
			"views" : 0,
			"objects" : 0,
			"avgObjSize" : 0,
			"dataSize" : 0,
			"storageSize" : 0,
			"totalSize" : 0,
			"indexes" : 0,
			"indexSize" : 0,
			"scaleFactor" : 1,
			"fileSize" : 0,
			"fsUsedSize" : 0,
			"fsTotalSize" : 0,
			"ok" : 1
		},
		"shard_repl_set3/shard3_node1:27017,shard3_node2:27017,shard3_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 1,
			"views" : 0,
			"objects" : 325716,
			"avgObjSize" : 1291.216750174999,
			"dataSize" : 420569955,
			"storageSize" : 64290816,
			"freeStorageSize" : 401408,
			"indexes" : 1,
			"indexSize" : 9809920,
			"indexFreeStorageSize" : 385024,
			"totalSize" : 74100736,
			"totalFreeStorageSize" : 786432,
			"scaleFactor" : 1,
			"fsUsedSize" : 20678135808,
			"fsTotalSize" : 62725623808,
			"ok" : 1
		},
		"shard_repl_set/shard1_node1:27017,shard1_node2:27017,shard1_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 0,
			"views" : 0,
			"objects" : 0,
			"avgObjSize" : 0,
			"dataSize" : 0,
			"storageSize" : 0,
			"totalSize" : 0,
			"indexes" : 0,
			"indexSize" : 0,
			"scaleFactor" : 1,
			"fileSize" : 0,
			"fsUsedSize" : 0,
			"fsTotalSize" : 0,
			"ok" : 1
		},
		"shard_repl_set4/shard4_node1:27017,shard4_node2:27017,shard4_node3:27017" : {
			"db" : "taxiDb",
			"collections" : 0,
			"views" : 0,
			"objects" : 0,
			"avgObjSize" : 0,
			"dataSize" : 0,
			"storageSize" : 0,
			"totalSize" : 0,
			"indexes" : 0,
			"indexSize" : 0,
			"scaleFactor" : 1,
			"fileSize" : 0,
			"fsUsedSize" : 0,
			"fsTotalSize" : 0,
			"ok" : 1
		}
	},
	"objects" : 325716,
	"avgObjSize" : 1291,
	"dataSize" : 420569955,
	"storageSize" : 64290816,
	"totalSize" : 74100736,
	"indexes" : 1,
	"indexSize" : 9809920,
	"scaleFactor" : 1,
	"fileSize" : 0,
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1641308077, 13),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1641308077, 1)
}

#### To shard this data following actions need to be performed:
 - go to mongo: `mongo mongodb://localhost:60000`
 - switch to taxiDb: `mongos> use taxiDb`
 - verfity the data is imported to one of the shards: `mongos> db.stats()`
 - enable sharding on db: `mongos> sh.enableSharding("taxiDb")`
 - enable sharding for the newly created collection: `mongos> sh.shardCollection("taxiDb.londonPostcodes", {"_id":1})`

`mongos> sh.enableSharding("taxiDb")`<br>
{
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1641569238, 1),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1641569238, 1)
}
`mongos> sh.shardCollection("taxiDb.londonPostcodes", {"_id":1})`<br>
{
	"collectionsharded" : "taxiDb.londonPostcodes",
	"ok" : 1,
	"$clusterTime" : {
		"clusterTime" : Timestamp(1641569520, 27),
		"signature" : {
			"hash" : BinData(0,"AAAAAAAAAAAAAAAAAAAAAAAAAAA="),
			"keyId" : NumberLong(0)
		}
	},
	"operationTime" : Timestamp(1641569520, 23)
}

#### Now we can see that the collection is sharded across 4 replica sets
`mongos> db.printShardingStatus()`<br>
<b>--- Sharding Status ---</b> <br>
  sharding version: {
  	"_id" : 1,
  	"minCompatibleVersion" : 5,
  	"currentVersion" : 6,
  	"clusterId" : ObjectId("61d315727807d7cbd4ec0f68")
  }<br>
  shards:<br>
        {  "_id" : "shard_repl_set",  "host" : "shard_repl_set/shard1_node1:27017,shard1_node2:27017,shard1_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641225152, 2) }
        {  "_id" : "shard_repl_set2",  "host" : "shard_repl_set2/shard2_node1:27017,shard2_node2:27017,shard2_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641227776, 1) }
        {  "_id" : "shard_repl_set3",  "host" : "shard_repl_set3/shard3_node1:27017,shard3_node2:27017,shard3_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641227760, 2) }
        {  "_id" : "shard_repl_set4",  "host" : "shard_repl_set4/shard4_node1:27017,shard4_node2:27017,shard4_node3:27017",  "state" : 1,  "topologyTime" : Timestamp(1641227797, 1) }<br>
  active mongoses:<br>
        "5.0.5" : 1<br>
  autosplit:<br>
        Currently enabled: yes<br>
  balancer:<br>
        Currently enabled: yes
        Currently running: no
        Collections with active migrations: 
                taxiDb.londonPostcodes started at Fri Jan 07 2022 17:34:22 GMT+0200 (EET)
        Failed balancer rounds in last 5 attempts: 0
        Migration results for the last 24 hours: 
                2 : Success<br>
  databases:<br>
        {  "_id" : "config",  "primary" : "config",  "partitioned" : true }<br>
                config.system.sessions
                        shard key: { "_id" : 1 }
                        unique: false
                        balancing: true
                        chunks:
                                shard_repl_set	256
                                shard_repl_set2	256
                                shard_repl_set3	256
                                shard_repl_set4	256
                        too many chunks to print, use verbose if you want to force print<br>
        {  "_id" : "taxiDb",  "primary" : "shard_repl_set3",  "partitioned" : true,  "version" : {  "uuid" : UUID("e13c509b-a04e-4699-825a-cd083fe4a6fe"),  "timestamp" : Timestamp(1641306656, 1),  "lastMod" : 1 } }<br>
                <b>taxiDb.londonPostcodes</b><br>
                        <b>shard key: { "_id" : 1 }</b><br>
                        unique: false<br>
                        <b>balancing: true<br>
                        chunks:</b><br>
                                shard_repl_set	1
                                shard_repl_set2	1
                                shard_repl_set3	5<br>
                        { "_id" : { "$minKey" : 1 } } -->> { "_id" : ObjectId("61d45a254bd5a323af789cb1") } on : shard_repl_set Timestamp(2, 0)
                        { "_id" : ObjectId("61d45a254bd5a323af789cb1") } -->> { "_id" : ObjectId("61d45a2a4bd5a323af7967bf") } on : shard_repl_set2 Timestamp(3, 0) 
                        { "_id" : ObjectId("61d45a2a4bd5a323af7967bf") } -->> { "_id" : ObjectId("61d45a2f4bd5a323af7a32cd") } on : shard_repl_set3 Timestamp(4, 0) 
                        { "_id" : ObjectId("61d45a2f4bd5a323af7a32cd") } -->> { "_id" : ObjectId("61d45a334bd5a323af7ad897") } on : shard_repl_set3 Timestamp(1, 3) 
                        { "_id" : ObjectId("61d45a334bd5a323af7ad897") } -->> { "_id" : ObjectId("61d45a374bd5a323af7b7e62") } on : shard_repl_set3 Timestamp(1, 4) 
                        { "_id" : ObjectId("61d45a374bd5a323af7b7e62") } -->> { "_id" : ObjectId("61d45a3c4bd5a323af7c242d") } on : shard_repl_set3 Timestamp(1, 5) 
                        { "_id" : ObjectId("61d45a3c4bd5a323af7c242d") } -->> { "_id" : { "$maxKey" : 1 } } on : shard_repl_set3 Timestamp(1, 6) 

## Task 3. Generate more data
Use London addresses for simulations below.
Simulate taxi orders and store in base (with custom intensity).
Simulate taxi movement tracking and store in base (up to 10GB data).
Taxi without order are moving at nearest area.
Simulate taxi drive feedback and store in base.
Test the result.

In [1]:
import random
import string
import pymongo
import names

In [5]:
mongodb_url = "mongodb://localhost:60000"
client = pymongo.MongoClient(mongodb_url)
db = client['taxiDb']
users_collection = db['users']
drivers_collection = db['drivers']

In [6]:
db, db.list_collection_names()

(Database(MongoClient(host=['localhost:60000'], document_class=dict, tz_aware=False, connect=True), 'taxiDb'),
 ['driversLastPoints', 'londonPostcodes', 'drivers', 'users', 'rides'])

In [4]:
db['londonPostcodes'].find_one().keys()

dict_keys(['_id', 'Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing', 'Grid Ref', 'County', 'District', 'Ward', 'District Code', 'Ward Code', 'Country', 'County Code', 'Constituency', 'Introduced', 'Parish', 'Built up area', 'Built up sub-division', 'Lower layer super output area', 'Rural/urban', 'Region', 'Altitude', 'London zone', 'LSOA Code', 'MSOA Code', 'Middle layer super output area', 'Parish Code', 'Census output area', 'Constituency Code', 'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated', 'Nearest station', 'Distance to station', 'Postcode area', 'Postcode district', 'Police force', 'Water company', 'Plus Code', 'Average Income', 'Travel To Work Area', 'ITL level 2', 'ITL level 3'])

#### Let's create collections for users and drivers first

In [7]:
get_random_msisdn = lambda: '380'+''.join(random.choice(string.digits) for _ in range(9))


def populate_user_col(db_for_insert, docs_n=9000):
    users_col = db_for_insert['users']
    get_user_doc = lambda name, msisdn: {'name': name, 'phone_num': msisdn}
    docs = [get_user_doc(names.get_full_name(), get_random_msisdn()) for _ in range(docs_n)]
    users_col.insert_many(docs)


def populate_drivers_col(db_for_insert, docs_n=6000):
    drivers_col = db_for_insert['drivers']
    docs = []
    
    def get_driver_doc(*args):
        props = ['name', 'phone_num', 'rides_n', 'active_days_n', 'licence_plate']
        return dict(zip(props, args[0]))
    def get_random_plate_n():
        get_two_letters = lambda: ''.join(random.choice(string.ascii_uppercase) for _ in range(2))
        pt2 = ''.join(random.choice(string.digits) for _ in range(4))
        return ''.join([get_two_letters(), pt2, get_two_letters()])

    for _ in range(docs_n):
        name = names.get_full_name()
        msisdn = get_random_msisdn()
        rides_n = random.randint(0, 500)
        active_days_n = random.randint(0, 1200)
        license_plate = get_random_plate_n()
        docs.append(get_driver_doc([name, msisdn, rides_n, active_days_n, license_plate]))
    drivers_col.insert_many(docs)

In [8]:
%%time
populate_user_col(db)

CPU times: user 26.8 s, sys: 549 ms, total: 27.4 s
Wall time: 28.4 s


In [15]:
%%time
populate_drivers_col(db)

CPU times: user 18.3 s, sys: 394 ms, total: 18.7 s
Wall time: 19.7 s


In [8]:
users_collection.count_documents({}), drivers_collection.count_documents({})

(9000, 6000)

In [9]:
users_collection.find_one(), drivers_collection.find_one()

({'_id': ObjectId('61d8ac6497c34fc0eda333b3'),
  'name': 'Roy Morley',
  'phone_num': '380266783011'},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef60f'),
  'name': 'William Edelstein',
  'phone_num': '380978402593',
  'rides_n': 391,
  'active_days_n': 155,
  'licence_plate': 'XP6658FL'})

#### Let's assign random last point (postcode) for each driver

In [10]:
from tqdm.notebook import tqdm, trange

In [11]:
def throw_coin(db, coll_name, pipeline, col, forbidden_items: list):
    doc_col = list(db[coll_name].aggregate(pipeline))[0][col]
    while doc_col in forbidden_items:
        doc_col = list(db[coll_name].aggregate(pipeline))[0][col]
    return doc_col

postcode_agg = [{'$match': {'In Use?': 'Yes'}},
                    {'$sample': {'size': 1 }}]
get_random_postcode = lambda forbidden_items: throw_coin(db, 'londonPostcodes', postcode_agg, 'Postcode', forbidden_items) # too slow

In [12]:
def get_london_postcodes():
    in_use = []
    coords = []
    cursor = db['londonPostcodes'].find({'In Use?': 'Yes'})
    for document in cursor:
        in_use.append(document['Postcode'])
        coords.append((document['Latitude'], document['Longitude']))
    return in_use, coords

postcodes, postcodes_coords = get_london_postcodes()

In [13]:
def assign_last_postcode(db):
    rand_ps = []
    cursor = drivers_collection.find({})
    get_loc_doc = lambda obj_id, postcode: {'driver_id': obj_id, 'last_location': postcode}
    for doc in tqdm(cursor):
        driver_id = doc['_id']
        postcode = random.sample(postcodes, 1)[0]
        rand_ps.append(get_loc_doc(driver_id, postcode))
    db['driversLastPoints'].insert_many(rand_ps)

In [109]:
%%time
assign_last_postcode(db)

0it [00:00, ?it/s]

CPU times: user 25.6 s, sys: 259 ms, total: 25.9 s
Wall time: 26.2 s


In [11]:
drivers_loc_collection = db['driversLastPoints']
drivers_loc_collection.count_documents({}), drivers_loc_collection.find_one()

(6000,
 {'_id': ObjectId('61d9aaaed0f865e2297f24f1'),
  'driver_id': ObjectId('61d8b5e4d0f865e2297ef611'),
  'last_location': 'W9 3JR'})

#### Now we're ready to generate synthetic taxi rides
Information about an order should include:
 - driver's id
 - user's id
 - starting point (postcode)
 - ending point (postcode)
 - trip start time
 - trip end time<br><br>

Information about an order may also include:
 - driver's rating
 - user's rating
 - driver's review (categorical)
 - user's review (categorical)

In [14]:
import datetime
from bson.objectid import ObjectId

In [15]:
def choose_text_and_num(options):
    use_text_review = bool(random.randint(0, 1))
    text_rev, rating = None, None
    if use_text_review:
        option = options[random.randint(0, len(options)-1)]
        text_rev = option[0]
        rating = random.randint(option[1][0], option[1][1])
    else:
        rating = random.randint(1, 5)
    return text_rev, rating


def gen_review_for_driver():
    reviews = [
        ('noisy client', (1,3)),
        ('excellent client', (4,5)),
        ('gave good tips', (4,5)),
        ('used strong language', (1,3))
    ]
    return choose_text_and_num(reviews)


def gen_review_for_user():
    reviews = [
        ('excellent service', (4,5)),
        ('nothing special', (3,4)),
        ('clean car', (4,5)),
        ('poor driving skills', (1,2)),
        ('pleasant conversation', (4,5))
    ]
    return choose_text_and_num(reviews)


def random_dt(dt1, dt2):
    dt1_ride = random.random() * (dt2 - dt1) + dt1
    dt2_ride = random.random() * (dt2 - dt1_ride) + dt1_ride
    return dt1_ride, dt2_ride

In [16]:
def get_bbox(points):
    bot_left_x, bot_left_y = float('inf'), float('inf')
    top_right_x, top_right_y = float('-inf'), float('-inf')
    for x, y in points:
        bot_left_x = min(bot_left_x, x)
        bot_left_y = min(bot_left_y, y)
        top_right_x = max(top_right_x, x)
        top_right_y = max(top_right_y, y)
    return bot_left_x, bot_left_y, top_right_x, top_right_y


def find_coords_in_between(x1, y1, x2, y2):
    bot_left_x, bot_left_y, top_right_x, top_right_y = get_bbox([(x1, y1), (x2, y2)])
    suitable_coords = []
    for c in postcodes_coords:
        lat, lon = c[0], c[1]
        if (bot_left_x<lat<top_right_x) and (bot_left_y<lon<top_right_y):
            suitable_coords.append((lat, lon))
    if len(suitable_coords)>=2:
        points_n = random.randint(1, max([int(len(suitable_coords)/10), 2]))
        suitable_coords = random.sample(suitable_coords, points_n)
    suitable_coords = [(x1, y1)]+suitable_coords+[(x2, y2)]
    return suitable_coords

In [17]:
def generate_ride_interval(db, dt1, dt2, drivers_used: list, users_used: list):
    driver = throw_coin(db, 'drivers', [{'$sample': {'size': 1 }}], '_id', drivers_used)
    user = throw_coin(db, 'users', [{'$sample': {'size': 1 }}], '_id', users_used)

    starting_point = list(db['driversLastPoints'].find({'driver_id': driver}))[0]['last_location']
    ending_point = get_random_postcode([starting_point])
    starting_coords = postcodes_coords[postcodes.index(starting_point)]
    ending_coords = postcodes_coords[postcodes.index(ending_point)]
    coord_seq = find_coords_in_between(starting_coords[0], starting_coords[1], ending_coords[0], ending_coords[1])

    dt1_ride, dt2_ride = random_dt(dt1, dt2)
    
    include_driver_rating, include_user_rating = bool(random.randint(0, 1)), bool(random.randint(0, 1))
    text_rev_driver, rating_driver = None, None
    text_rev_user, rating_user = None, None
    if include_driver_rating:
        text_rev_driver, rating_driver = gen_review_for_user()
    if include_user_rating:
        text_rev_user, rating_user = gen_review_for_driver()
    
    result = {'driver_id': driver,
            'user_id': user,
            'dt_start': dt1_ride,
            'dt_end': dt2_ride,
            'trip_start_postcode': starting_point,
            'trip_end_postcode': ending_point,
            'path': coord_seq,
            'rating_driver': rating_driver,
            'rating_user': rating_user,
            'review_driver': text_rev_driver,
            'review_user': text_rev_user}
    return result


def synthetic_rides(db, dt_start, dt_end, no_usr_proba=0.3, n=1000):
    while dt_start<dt_end:
        intrvl_rides, intrvl_users = [], []
        intrvl_drivers, intrvl_end_locs = [], []
        intrvl_end = dt_start + datetime.timedelta(minutes=30)
        print(f'generating rides for {dt_start} - {intrvl_end}')
        for _ in trange(n):
            is_with_passanger = (random.random() > no_usr_proba)
            ride = generate_ride_interval(db, dt_start, intrvl_end, intrvl_drivers, intrvl_users)
            
            if not(is_with_passanger):
                trash = ['user_id', 'rating_driver', 'rating_user', 'review_driver', 'review_user']
                for col in trash:
                    ride[col] = None
            else:
                intrvl_users.append(ride['user_id'])
            
            ride = {k: v for k, v in ride.items() if v is not None}
            intrvl_rides.append(ride)
            intrvl_drivers.append(ride['driver_id'])
            intrvl_end_locs.append(ride['trip_end_postcode'])
        
        new_locs_upd = [pymongo.UpdateOne({'driver_id':intrvl_drivers[i]}, {'$set': {'last_location': intrvl_end_locs[i]}}) for i in range(n)]
        print('updating driversLastPoints')
        db['driversLastPoints'].bulk_write(new_locs_upd)
        print('inserting new rides')
        db['rides'].insert_many(intrvl_rides)
        
        dt_start = intrvl_end

In [22]:
%%time
d1 = datetime.datetime(2022, 1, 3, 11, 30)
d2 = datetime.datetime(2022, 1, 3, 12, 31)
synthetic_rides(db, d1, d2)

generating rides for 2022-01-03 11:30:00 - 2022-01-03 12:00:00


  0%|          | 0/1000 [00:00<?, ?it/s]

updating driversLastPoints
inserting new rides
generating rides for 2022-01-03 12:00:00 - 2022-01-03 12:30:00


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
db['rides'].count_documents({}), db['rides'].find_one()

(3000,
 {'_id': ObjectId('61daed64887ed079a8563d89'),
  'driver_id': ObjectId('61d8b5e4d0f865e2297efc05'),
  'user_id': ObjectId('61d8ac6497c34fc0eda33830'),
  'dt_start': datetime.datetime(2022, 1, 3, 10, 22, 11, 853000),
  'dt_end': datetime.datetime(2022, 1, 3, 10, 23, 55, 645000),
  'trip_start_postcode': 'SE9 6QL',
  'trip_end_postcode': 'EN1 1DF',
  'path': [[51.454232, 0.045673],
   [51.459523, -0.057593],
   [51.577107, -0.029263],
   [51.565145, -1.2e-05],
   [51.588489, -0.006306],
   [51.461649, -0.018605],
   [51.546024, 0.025368],
   [51.585658, -0.025571],
   [51.587036, -0.010772],
   [51.455769, 0.035378],
   [51.465195, 0.020191],
   [51.564851, -0.015941],
   [51.545032, -0.056434],
   [51.577027, -0.009609],
   [51.514345, -0.052913],
   [51.522835, -0.047577],
   [51.499839, -0.062104],
   [51.562703, -0.008718],
   [51.457412, -0.023855],
   [51.493095, -0.064839],
   [51.524895, 0.031898],
   [51.464085, 0.03808],
   [51.493562, 0.034105],
   [51.542268, -0.040544

Here I selected the drivers who had the most rides count and printed out the rides of one of them:

In [19]:
p = [{'$group': {'_id': '$driver_id', 
                'rides_cnt': {'$sum': 1}}},
     {'$sort': {'rides_cnt': -1}}]
rides_cnt = db['rides'].aggregate(p)

In [20]:
list(rides_cnt)[:8]

[{'_id': ObjectId('61d8b5e4d0f865e2297efba8'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef94e'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef6b5'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0670'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef6b8'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0352'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef8f2'), 'rides_cnt': 3},
 {'_id': ObjectId('61d8b5e4d0f865e2297f04ec'), 'rides_cnt': 3}]

In [22]:
cursor = db['rides'].find({'driver_id': ObjectId('61d8b5e4d0f865e2297efba8')}).sort([("dt_start", pymongo.ASCENDING)])
for i, doc in enumerate(cursor):
    print(f'-------- RIDE {i} ----------')
    doc['path'] = '...'
    print(doc)

-------- RIDE 0 ----------
{'_id': ObjectId('61daed64887ed079a8564080'), 'driver_id': ObjectId('61d8b5e4d0f865e2297efba8'), 'dt_start': datetime.datetime(2022, 1, 3, 10, 22, 46, 350000), 'dt_end': datetime.datetime(2022, 1, 3, 10, 29, 30, 768000), 'trip_start_postcode': 'NW8 8BW', 'trip_end_postcode': 'SW4 6JH', 'path': '...'}
-------- RIDE 1 ----------
{'_id': ObjectId('61daee2d887ed079a856442c'), 'driver_id': ObjectId('61d8b5e4d0f865e2297efba8'), 'dt_start': datetime.datetime(2022, 1, 3, 10, 50, 39, 514000), 'dt_end': datetime.datetime(2022, 1, 3, 10, 50, 50, 378000), 'trip_start_postcode': 'SW4 6JH', 'trip_end_postcode': 'KT2 6AL', 'path': '...'}
-------- RIDE 2 ----------
{'_id': ObjectId('61daef2b887ed079a8564689'), 'driver_id': ObjectId('61d8b5e4d0f865e2297efba8'), 'user_id': ObjectId('61d8ac6497c34fc0eda33530'), 'dt_start': datetime.datetime(2022, 1, 3, 11, 53, 38, 232000), 'dt_end': datetime.datetime(2022, 1, 3, 11, 59, 46, 692000), 'trip_start_postcode': 'KT2 6AL', 'trip_end_p

In [35]:
cursor = db['rides'].find({'driver_id': ObjectId('61d8b5e4d0f865e2297f081f')}).sort([("dt_start", pymongo.ASCENDING)])
for i, doc in enumerate(cursor):
    print(f'-------- RIDE {i} ----------')
    doc['path'] = '...'
    print(doc)

-------- RIDE 0 ----------
{'_id': ObjectId('61daef2b887ed079a8564646'), 'driver_id': ObjectId('61d8b5e4d0f865e2297f081f'), 'user_id': ObjectId('61d8ac6497c34fc0eda34f41'), 'dt_start': datetime.datetime(2022, 1, 3, 11, 43, 16, 946000), 'dt_end': datetime.datetime(2022, 1, 3, 11, 49, 0, 591000), 'trip_start_postcode': 'KT1 1SY', 'trip_end_postcode': 'RM1 1EG', 'path': '...', 'rating_user': 5, 'review_user': 'excellent client'}


In the output of the above cell we can see that the driver with id 61d8b5e4d0f865e2297ef882 have completed 4 rides - only 2 of them with passengers (featuring user_id)

## Task 4. Analyze the rides
Analyse drivers and feedbacks according to Lab 5. Present top 100 drivers rating-vise.

In [45]:
p = [{'$group': {'_id': '$driver_id', 
                'avg_rating': {'$avg': '$rating_driver'},
                'rides_cnt': {'$sum': 1}}},
     {'$match': {'rides_cnt': {'$gt': 1}}},
     {'$sort': {'avg_rating': -1}},
     {'$limit': 100}]
top_rating = db['rides'].aggregate(p)

In [46]:
list(top_rating)

[{'_id': ObjectId('61d8b5e4d0f865e2297efe63'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0842'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0367'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef91a'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297efafd'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297efca2'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0091'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0b17'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297ef98c'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f0214'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f04bc'),
  'avg_rating': 5.0,
  'rides_cnt': 2},
 {'_id': ObjectId('61d8b5e4d0f865e2297f00cb

In [134]:
client.close()